## Interacting with Azure Open AI API

In [1]:
import openai
import json
import os
from dotenv import load_dotenv


In [2]:
load_dotenv("openai.env")


True

In [3]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY") , 
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE") 
)

In [22]:
msgs = [
        {"role": "system", "content": """You are an AI chatbot built by Vodafone. You can questions on scheduling appointments."""},
        {"role": "user", "content": "what's 2 + 5"}
    ]

In [25]:
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "stop",
            "description": "This function gracefully concludes an interaction.",
            "parameters":
            {
                "type": "object",
                "properties":
                {
                    "arrangeAppointment":
                    {
                        "type": "string",
                        "description": "Accepts either canceling or rescheduling an appointment.",
                        "enum": ["cancelAppointment", "rescheduleAppointment", "keepAppointment"]
                    }
                },
                "required":
                []
            }
        }
    },
    {
        "type":"function",
        "function":
        {
            "name": "addNumbers",
            "description": "This function adds two numbers",
            "parameters":
            {
                "type": "object",
                "properties":
                {
                    "x":
                    {
                        "type": "number",
                        "description": "first number to be added"
                    },
                    "y":
                    {
                        "type": "number",
                        "description": "second number to be added"
                    }
                }
            }
        }
    }
    
]

In [26]:
chat_completion = client.chat.completions.create(
    model="gpt-4-Turbo", # model = "deployment_name".
    messages=msgs,
    temperature=0,
    top_p=0.95,
    presence_penalty=0,
    frequency_penalty=0,
    tools=tools
)

In [ ]:
chat_completion

In [27]:
json.loads(chat_completion.model_dump_json(indent=2))['choices'][0]['message']

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_IV2ZCEzAKd2LXz8LUJZGGqb5',
   'function': {'arguments': '{"x":2,"y":5}', 'name': 'addNumbers'},
   'type': 'function'}]}

In [29]:
response_message = chat_completion.choices[0].message
tool_calls = response_message.tool_calls


In [30]:
tool_calls

[ChatCompletionMessageToolCall(id='call_IV2ZCEzAKd2LXz8LUJZGGqb5', function=Function(arguments='{"x":2,"y":5}', name='addNumbers'), type='function')]

In [31]:
def addNumbers(a, b):
    return a+b

In [32]:
available_functions = {
            "addNumbers": addNumbers,
        } 

In [33]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    print(function_name)
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        a=function_args.get("x"),
        b=function_args.get("y"),
    )

addNumbers


In [34]:
function_response

7